In [1]:
import heapq
from collections import deque

In [2]:
def part_1(maze_str):
    """
    Given a maze as a multiline string, computes and returns the lowest score
    achievable to go from 'S' tile to 'E' tile.
    Movement and cost rules:
      - Move forward into a non-wall tile costs +1 point.
      - Turning left or right 90 degrees costs +1000 points.
    The Reindeer starts on 'S' facing East.
    """
    maze = maze_str.strip('\n').split('\n')
    rows = len(maze)
    cols = len(maze[0])

    # Directions: N=0, E=1, S=2, W=3
    directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    start_dir = 1  # Facing East at the start

    # Find start and end
    start, end = None, None
    for r in range(rows):
        for c in range(cols):
            if maze[r][c] == 'S':
                start = (r, c)
            elif maze[r][c] == 'E':
                end = (r, c)

    # dist[r][c][d]: best cost to reach (r,c) facing direction d
    dist = [[[float('inf')] * 4 for _ in range(cols)] for __ in range(rows)]
    dist[start[0]][start[1]][start_dir] = 0
    pq = []
    heapq.heappush(pq, (0, start[0], start[1], start_dir))

    while pq:
        cost, r, c, d = heapq.heappop(pq)
        if cost > dist[r][c][d]:
            continue

        # Check if end reached
        if (r, c) == end:
            return cost

        # Move forward
        dr, dc = directions[d]
        nr, nc = r + dr, c + dc
        if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#':
            new_cost = cost + 1
            if new_cost < dist[nr][nc][d]:
                dist[nr][nc][d] = new_cost
                heapq.heappush(pq, (new_cost, nr, nc, d))

        # Turn left
        left_d = (d - 1) % 4
        new_cost = cost + 1000
        if new_cost < dist[r][c][left_d]:
            dist[r][c][left_d] = new_cost
            heapq.heappush(pq, (new_cost, r, c, left_d))

        # Turn right
        right_d = (d + 1) % 4
        new_cost = cost + 1000
        if new_cost < dist[r][c][right_d]:
            dist[r][c][right_d] = new_cost
            heapq.heappush(pq, (new_cost, r, c, right_d))

    return None

def shortest_path_with_distances(maze):
    maze = maze.strip('\n').split('\n')
    rows = len(maze)
    cols = len(maze[0])

    directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    start_dir = 1  # Start facing East

    # Find start and end
    start = None
    end = None
    for r in range(rows):
        for c in range(cols):
            if maze[r][c] == 'S':
                start = (r, c)
            elif maze[r][c] == 'E':
                end = (r, c)

    # dist[r][c][d]: best cost to reach (r,c) facing direction d
    dist = [[[float('inf')] * 4 for _ in range(cols)] for __ in range(rows)]
    dist[start[0]][start[1]][start_dir] = 0

    pq = []
    heapq.heappush(pq, (0, start[0], start[1], start_dir))

    while pq:
        cost, r, c, d = heapq.heappop(pq)
        if cost > dist[r][c][d]:
            continue

        if (r, c) == end:
            # We found a path to the end, but keep going to ensure all shortest distances are found
            # because we need all shortest path states.
            pass

        # Move forward
        dr, dc = directions[d]
        nr, nc = r + dr, c + dc
        if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#':
            new_cost = cost + 1
            if new_cost < dist[nr][nc][d]:
                dist[nr][nc][d] = new_cost
                heapq.heappush(pq, (new_cost, nr, nc, d))

        # Turn left
        left_d = (d - 1) % 4
        new_cost = cost + 1000
        if new_cost < dist[r][c][left_d]:
            dist[r][c][left_d] = new_cost
            heapq.heappush(pq, (new_cost, r, c, left_d))

        # Turn right
        right_d = (d + 1) % 4
        new_cost = cost + 1000
        if new_cost < dist[r][c][right_d]:
            dist[r][c][right_d] = new_cost
            heapq.heappush(pq, (new_cost, r, c, right_d))

    return maze, dist, start, end, directions

def part_2(maze_str):
    maze, dist, start, end, directions = shortest_path_with_distances(maze_str)
    rows = len(maze)
    cols = len(maze[0])

    # Find the minimal cost to reach the end in any direction
    end_cost = min(dist[end[0]][end[1]][d] for d in range(4))
    if end_cost == float('inf'):
        # No path
        return maze, 0

    # We need to find all states (r,c,d) that are part of at least one shortest path.
    # We'll do a backward search from all end states that have dist = end_cost.
    # A state (r,c,d) is on a shortest path if there exists a next state (nr,nc,nd)
    # reachable by one of the forward moves whose dist differs by the exact cost increment.

    # Let's build a graph backward by checking conditions:
    # From (r,c,d), previous states can be:
    #   1) Moving forward: (r - dr, c - dc, d) if dist[r - dr][c - dc][d] + 1 = dist[r][c][d]
    #   2) Turning left: (r, c, (d+1)%4) if dist[r][c][(d+1)%4] + 1000 = dist[r][c][d]
    #   3) Turning right: (r, c, (d-1)%4) if dist[r][c][(d-1)%4] + 1000 = dist[r][c][d]

    # We'll start from all end states that have dist = end_cost and run a BFS/DFS backward.
    best_path_states = set()
    queue = deque()

    # Initialize queue with all end direction states at minimal cost
    for d in range(4):
        if dist[end[0]][end[1]][d] == end_cost:
            queue.append((end[0], end[1], d))
            best_path_states.add((end[0], end[1], d))

    while queue:
        r, c, d = queue.popleft()
        current_cost = dist[r][c][d]

        # Check forward predecessor
        dr, dc = directions[d]
        pr, pc = r - dr, c - dc
        if 0 <= pr < rows and 0 <= pc < cols and maze[pr][pc] != '#':
            # If coming from forward move:
            # dist[pr][pc][d] + 1 == dist[r][c][d]
            if dist[pr][pc][d] + 1 == current_cost:
                if (pr, pc, d) not in best_path_states:
                    best_path_states.add((pr, pc, d))
                    queue.append((pr, pc, d))

        # Check turn left predecessor
        # If (r,c,d) was obtained by a left turn from (r,c,d+1)
        pd_left = (d + 1) % 4
        if dist[r][c][pd_left] + 1000 == current_cost:
            if (r, c, pd_left) not in best_path_states:
                best_path_states.add((r, c, pd_left))
                queue.append((r, c, pd_left))

        # Check turn right predecessor
        # If (r,c,d) was obtained by a right turn from (r,c,d-1)
        pd_right = (d - 1) % 4
        if dist[r][c][pd_right] + 1000 == current_cost:
            if (r, c, pd_right) not in best_path_states:
                best_path_states.add((r, c, pd_right))
                queue.append((r, c, pd_right))

    # Now we have all states on shortest paths. 
    # We need to find all distinct tiles (r,c) that are in best_path_states.
    best_path_tiles = set((r,c) for (r,c,d) in best_path_states)

    return maze, best_path_tiles

In [3]:
maze_input = """
#############################################################################################################################################
#.....#.........#.#.........#.........#.......#.................................#...#.....#.......#.......#.....#.......#..................E#
#.###.#.#.#####.#.#.#####.###.#####.#.#.#####.#.#########.#.###.#.###.#####.#.#.#.###.#.###.#.#.#.#.#.###.#.###.#.#.###.#.#########.#####.###
#...#...#...#.....#.....#.#...#.....#.#.......................#.................#.....#.....#.#...#.#.#...#.#.#...#.#...#.#...#.........#...#
###.#.###.#.#.#########.#.#.###.#####.#.#.#.#.#.###.#.###.###.###.#.#.#.#.#.#.#######.###.###.#####.#.#####.#.#####.###.#.#.#.###.#.###.###.#
#...#...#.........................#...#.#.#...#...#.#.......#.#...#...#.#.#.#.....#.....#...#.....#.#.............#...#.#.#.#...#.#...#...#.#
#.#####.#.#.#####.#.###.#####.###.#.###.#.#.###.#.#.#.###.###.#.#.#.###.#.#.#.#.#.#.#.#.#####.#.#.#.###########.#####.###.#.###.#.###.#####.#
#.#.....#.#.#.....#.....#...#...#.......#.#.#...#...#...#.#...#.#...#...#.#.....#...#.#.......#.#.#.#.....#...#.#...#.......#...#.#...#...#.#
#.#.###.#.###.###########.#.#######.#####.###.#.#########.#.#####.###.###.#.#.#######.#######.#.#.#.#.###.#.#.###.#.#########.#.###.#.#.#.#.#
#.#.....#...#.............#...#.......#...#.....................#.#.#.#.#.........#...#.....#.#...........#.#.....#...................#.#.#.#
#.###.#.###.#####.#.#.#######.#.###.#.#.#.#.#######.###.#.#####.#.#.#.#.#.#.#.###.#.#.#.#####.#.#.#.###.#.#.###########.#####.#.###.###.#.#.#
#...#.#...#.........#.#...#...#...#.....#.#...#...#.#...#.....#.....#.#.......#...#.#...#...#.#.#.........#...#.#.......#...#...#...#...#...#
###.###.#.###########.###.#.#####.#.#.#######.#.#.#.###.###.#.#.#.###.#.#.#.###.###.#.###.#.#.#.#############.#.#.#######.#.#.#.###.#.###.#.#
#...#.......#...#.....#...#.#.....#...#.....#...#.#...#.....#.#.....#.#.#.#.#...#.........#...#...#.........#.#...........#.......#.#...#...#
#.###.#.###.###.#.#####.###.#.#####.###.###.#####.###.#####.#####.#.#.###.#.#.###.###.###.###.###.#.#######.#.#.#########.###.#.#.#.###.#####
#.#...#.........#.#.........#.#.......#...#.....#.....#...#.#...#...#.....#.#...#.....#.....#.#...#.#.....#.#.#.#.#.....#.#...#...#...#.....#
#.###.#.#######.#.#####.#####.#.###.#.###.#.###.#.#####.#.###.#.#.#.#######.###.###.###.###.#.#####.#.###.#.#.#.#.#.###.###.#.#.###.#######.#
#...#.....#...#.#.....#.#...#...#...#...#.#.#...#.#.....#.....#.#.......#.....#.......#.#.#.#.....#.#...#.....#...#.#.#.....#.#.#...#.....#.#
###.#.#.#.#.#.#.#####.#.#.#.#####.#.#.###.#.#####.#.###########.#######.#.#.#.#######.#.#.#.#####.#.###.###########.#.#######.#.#####.###.#.#
#.#.#...#.#.#...#...#.#.#.#.......#...#...#.......#...#.......#.#.......#.#.......#.#.#...#.....#.#.#...#.....#.....#.......#.#.......#...#.#
#.#.#####.#######.#.#.#.#.#.#####.###.#.#####.#######.#.###.###.#.#######.#.#.###.#.#.###.#####.#.#.#.###.###.#.#########.#.#.#.#######.###.#
#.................#.#.#.#.#.#...#.....#...#.#.........#.#.......#.#.......#.#...#.#.#.....#.#...#.#.#.#...#.#...#.....#...#.#.........#.#...#
#.#####.#.#######.#.#.#.#.###.#.#########.#.#.#########.#.#.###.#.#.#######.#.###.#.#.###.#.#.###.#.###.###.#######.#.#.###.###.#####.#.#.#.#
#.....#.#.......#.#.#.#...#...#.....#.......#...#.....#.#.........#.....#...#.....#.#.......#.#.#.#...#...#.......#.#...#.....#.....#.#.....#
#.#.###.#####.#.###.#.#####.#######.#.#.#######.#.#####.#.#.#####.#####.###.#.#####.#######.#.#.#.#.#.###.#.###.#.#.#.#####.###.#.#.#.#####.#
#.....#...#...#.....#...#...#...#...#.#.#.......#.#.....#.#.#...#.....#.....#.#.......#.....#.#.#.......#.#...#.....#.#...#.....#.#.#.....#.#
#.#.#.###.#.#######.###.#.#####.#.###.#.#.#######.#.#.#.#.#.###.#####.#######.#######.#.###.#.#.#######.#.###.#.#####.#.#.#.#.###.#.#####.#.#
#.#.#...#...#.....#...#.#.#...#.#.......#.#...#.....#...#.#...#.....#.......#...#...#.#.......#.....#...#...#.#.#...#.#.#...#.....#.....#.#.#
#.#.#.#########.###.###.#.#.#.#.#####.###.#.#.#.#############.#.###.#######.###.#.#.#.#.###.#######.#.#.###.#.###.#.###.#####.#########.#.###
#...#...........#...#...#...#...#...#.....#.#.#.#.............#.#...#.....#...#.#.#...#...#.#.....#...#...#.#...#.#...#.#...#.#.......#.#...#
###.###.#########.###.###.#####.#.###.#######.#.###.#####.#.#####.###.#.#####.#.#.#######.###.###.###.#.#.#.#.#.#.###.#.#.#.#.#.#####.#.###.#
#.....#.......#...#...#.......#.#...#.#.......#...#.#.#...#.#...#.....#.......#.#...#...#...#...#...#.#.#.#.#.#.....#...#.#.#...#.#...#.#...#
#.#.#####.###.#.###.###.#.#####.#.#.#.###.###.###.#.#.#.#.###.#.#.#####.###.#.#.###.#.#.###.#.#####.#.#.###.#.###########.#.#.###.#.#####.#.#
#.#.....#...#...#...#...#.#.....#.#.#...#.#.....#.#.#.#.#.#...#.......#.......#.......#...#.#.#.....#...#...#...#.........#.#.....#.....#.#.#
###.###.#####.###.###.###.#.#######.###.#.#######.#.#.#.#.#.#########.#.#########.#.#####.#.#.#.###.#####.###.#.#.#.#####.#.###########.#.###
#...#.#.....#...#...#.#...#...#...#.#...#.........#...#.#.#.#.......#.#.#.....#...#.....#.#...#.........#.#...#.#.#.#.....#.........#...#...#
#.###.#####.###.###.#.#.#####.#.#.#.#.###.#############.###.###.#####.#.#.###.#.#####.###.#############.#.#.#.###.#.#.###.#########.#.#.###.#
#...#.......#.....#...#.........#.#.#.#...#...#.......#.#...#.......#.#.#...#.#.#.....#...#.....#.#...#.#.#.#.#...#.#...#...#.#.......#.#...#
###.#.###########.###############.#.#.#.###.#.#.###.#.#.#.###.#.#.#.#.###.#.#.#.#.#####.#.#.###.#.#.#.#.#.#.#.#.#####.#.###.#.#.#########.###
#...#...........#.#.............#.#.#.#...#.#...#.#.#...#.#...#.#.#...#...#.#...#.#...#.#...#.#.#...#.#...#...#.#.......#...#.....#.....#...#
#.#############.#.#.#####.#####.#.#.#.#.#.#.#####.#.#####.###.###.###.#.###.#######.#.#.#####.#.#####.#####.#.#.#.#######.#######.#.###.###.#
#.#...#.........#.#...#.#.....#.#...#.#.#.#.......#.....#.#...#...#...#.....#.......#...#...#...........#.....#.#.#.....#.......#.#.#.#.#...#
#.###.#.#########.###.#.#.###.#.#.###.#.#.###.#.#######.#.#.#.#.###.#######.###.#########.###.###.#####.#.###.#.#.#.#.#.#######.#.#.#.#.#.###
#...#.#.........#.#...#.#...#...#.#...#.#...#.#.......#...#.#...#.#.#.....#.....#...#.........#...#...#.#.#.#.#.....#.#...#.....#.#.....#...#
#.#.#.###.###.###.#.###.###.#######.###.###.###.#####.#.###.#.###.#.#.###.#######.###.#########.#.#.###.#.#.#.#.#####.#.###.#.###.#.#.#####.#
#.#.#...#...#.....#.#...#...#.......#...........#.#...#.....#.#.....#...#...#...#.....#.........#.#...#.#.#...#.......#.#...#.#.....#.......#
#.#.###.###.#.#####.#.#.#.#.#####.#######.#######.#.###.#####.#.#########.#.#.###.#####.#########.###.#.#.#.#######.#####.###.#.#.#########.#
#.#...#.....#.....#.#.#.#.......#.......#.#.....#...#...#.....#...#.......#...#...#.....#...#.......#.....#.#.....#.#...#...#...............#
#####.#####.#.#####.#.#########.#######.###.###.#.#######.#######.###.#.#####.#.###.#####.#.#.#.###.#######.###.#.###.#.#.#.#.###.#.#.###.###
#...#.....#.#.#.....#.........#...#.....#.....#.#.........#.....#.#.....#...#.#.#.........#...#...#.#...........#.....#.#.#.#.#...#.....#...#
#.#.#####.#.###.###.#.#.#.#.#.###.#.#####.###.#.#####.#########.#.#.###.#.#.#.#.#####.###########.#.#.#.###############.#.#.#.###.#######.#.#
#.#.......#.#...#...#.#.#...#...#.#.#.....#...#.....#.#.......#.....#...#.#...#.......#.....#.#...#.#.#.......#.......#...#.#.....#.....#.#.#
#.#.#######.#.###.#.###.#.#.#####.#.#.#######.#####.#.#.#####.#.#########.###.#########.###.#.#.###.#.#######.###.#.#######.#####.#.###.###.#
#.#.#.....#...#...#...#.#.#.......#.#.......#.....#.#.#.#...#.#.#.......#.#...#.......#...#...#.#.....#.....#...#.#...............#.#.......#
#.###.###.#####.#####.#.#.#########.#######.#####.#.###.#.###.###.#####.#.#.###.#.#####.#.#####.#######.###.###.#.###############.#.#########
#.....#.#.......#.....#.#...#.......#.#.....#...#.#.....#...#.#...#...#.#.#.....#.......#.............#.#.#.....#.#...........#...#...#.....#
#.#####.#########.#.###.###.#.#####.#.#.#####.#.#.#########.#.#.#####.#.#.#####.#####################.#.#.#######.#######.###.#.#####.#.#.#.#
#.....#.......#...#.#.......#.#.......#.#.....#.#.#.......#.#...#.....#...#...#...#.................#...#.....#.#.#.......#...#.....#.....#.#
#####.#.#.###.#.#.#.#.#####.#.#######.#.#.#.#.#.#.#.###.#.#.#####.###.#####.#.#####.###############.#######.#.#.#.#.#######.#####.#######.#.#
#...#.#.#.#...#.#.....#...#...#...#...#.#...#.#.#...#.#.#.....#...#.....#...#...#...#.......#.....#.......#.#...#.#.#.......#.....#.......#.#
###.#.#.#.###.#.#######.#.#####.#.#####.#####.#.#.###.#.#####.#.#.#####.#.#####.#.###.###.#.#####.#######.#.###.#.#.###.#.#.#.#.###.#####.#.#
#...#.#.#...#.#.........#.....#.#.............#.#.#...#.#...#...#...#...#.....#.#.#...#.#.#.............#.#...#.#.#...#.#.#...#...#.#...#...#
#.###.#####.#################.#.#####.#######.#.#.#.#.#.#.###.#####.#.#######.#.#.#.###.#.###.#######.###.###.###.###.#.#.#.#####.#.#.#.#.#.#
#...#.....#...#...#...........#.#.......#.......#...#.#.#...#.#...#.#...#...#.#...#.....#.#...#...........#...#...#...#.#.#...#...#.#.#...#.#
#.#.#####.#.#.#.#.#.###########.#.#.#####.#############.#.#.#.#.#.#.#.#.#.#.#.###.###.###.#####.###########.###.###.###.#.#####.###.#.#.###.#
#.#.......#.#...#.#.............#.#.#...#.#.............#.#.#.#.#...#.....#.#...#.....#...#.....#.......#.......#...#...#.#.....#...#.#.....#
#.#########.###.#.###############.###.#.#.###.#######.#####.#.#.#####.#########.#.#####.###.#####.###.###.#######.###.#.#.#.###.#.#####.#####
#.#.#.....#.#.#...#.............#...#.#.#.#...#...#.....#...#...#.....#.........#.....#.#...#...#.#.#.............#...#.#.#.#...#.#...#...#.#
#.#.#.#.###.#.#.###.#####.#.#.#####.#.#.#.#.###.###.#.#.#.#.#.###.###.#.#.#########.###.#.###.#.#.#.###############.#.#.#.#.#####.#.#.###.#.#
#.#...#.......#.......#...#...#...#...#...#.#...#.....#...#...#...#.....#.......#.......#.#...#.#...#.........#.#...#.#.#.#.#...#...#.#.#...#
#.###.#######.#######.#.###.###.#.#########.#.###.###.#####.###.#########.#####.#.###.###.###.#.#####.#######.#.#.###.###.#.#.#.#####.#.###.#
#...#.#...#.#.....#.#.#...#...#.#...#.#.....#.......#.........#...#...#.#.......#.#.#.....#...#...#...#.....#.#.#.#.#.....#...#.#...#.#.....#
###.#.#.#.#.#.###.#.#.###.###.#.#.#.#.#.###.#.###################.#.#.#.#.#.###.#.#.###.###.#####.#.###.#####.#.#.#.#######.###.#.#.#.#.#####
#...#...#...#.#.#.#.#...#...#.#.#.#.#.....#.#.#.........#...#...#...#.#.#.#...#.#.#...#.....#...#...#.........#...#.......#...#.#.#...#...#.#
#.#########.#.#.#.#.###.###.###.#.#.#######.###.#######.#.#.#.#.#.###.#.#####.#.#.#.#.#.###.#.#.###.#.#########.#########.#.###.#.#######.#.#
#.........#.#.#.#.#.......#.......#.......#.....#.#...#.#.#.#.#.......#.#...#.#...#.#.#...#.#.#.......#.........#.......#...#...#.....#.....#
#########.#.#.#.#.#########.#####.#######.#.#.###.#.#.#.#.#.#.#######.#.#.#.#.###.#.#.#####.#.#########.#####.###.#.#.#.#####.#######.#####.#
#.......#.#.#.#...........#.....#.#.....#...#.....#.#.#...#.#.....#...#...#.#.......#.#...#.#.#.....#...#...#...#.#.#.#.....#.#.....#.....#.#
#.#######.#.#.#######.###.#.#####.#.#######.#######.#.#.#.#.#####.###.#####.#########.#.#.#.#.#.###.#.#.#.#.###.###.#.#####.#.#.#.#######.#.#
#.........#.#.....#.......#.#.....#.#.......#.......#...#.#.....#...#.....#.......#...#.#.#.#...#.#.#.#.#.#.#.......#.......#...#.....#...#.#
#.#########.#.#.#.#####.#####.#####.#.#######.#########.#.#####.#.#.#############.#.###.#.#.#####.#.#.#.#.#.#######.#######.#.#####.###.#####
#...#.....#.....#.....#.......#.#.....#...#.....#.....#.#.#...#.#.#...........#...#.....#...#.......#.#.#.#...#.....#.....#.#.#...#.#...#...#
###.#.#.#####.#.#####.#########.#.#######.#.###.#.#####.#.#.###.###########.#.#.#############.#.#####.#.#.###.#####.#.###.#.###.#.#.#.###.#.#
#...#.#.............#...#...............#...#...#.........#...............#...#.#.....#.....#.#...#.....#.#...#...#.#...#.#.....#.#.#.....#.#
#.###.#######.#####.###.#####.#.#######.#####.###.#######.#.###########.#.#.#.#.###.#.#.###.#.#.#.#.#####.#.#.#.#.###.#.#.#######.#.#######.#
#...#.....#...........#.............#.........#.#.#.....#.#.#...........#...#.#.....#.#.#...#...#.#.....#.#...#.#.....#.#.#.#.............#.#
###.###.###.#.###.###.#######.#####.#.#########.#.###.#.#.#.#.#.#.#####.#############.#.#######.#.###.###.#.###.#####.#.#.#.#.#############.#
#...#...#...#...#...#.#.....#.#.....#.#...#...#.......#.#.#.#.#.#.#.....#...........#.#.#.......#.#.#.#...#.....#...#.#...#...#.#.........#.#
#.#######.#.#.#.###.#.#.###.###.#####.#.#.#.#.#.#########.###.#.#.#.###.#.###.#######.#.#.#####.#.#.#.#.#####.###.#.#.#.###.###.#.#######.#.#
#...........#.#...#.#.#.#.#...#.#...#.#.#...#.#.#.#...#...#...#.#.#.#.#.#.#.#.#.......#.#.....#...#...#.#.....#...#.#...#.......#.#.....#...#
#.#.#.#.###.#.#.#.#.###.#.###.#.#.#.###.#.#.#.#.#.#.#.#.###.###.#.#.#.#.#.#.#.#.#######.#####.#########.#.#####.###.#.#######.###.###.#.###.#
#...#...#.#.....#.#.....#.#.....#.#.....#.#.#.....#.#.#.......#...#.#.#.#...#...#...#.......#.#.........#.#...#.#...#.......#.#...#...#.#...#
#########.#.###.#.#######.#.#####.#######.#.#####.#.#.###########.#.#.#.###.#####.#.#.#.#####.#.###.#.#####.#.#.#.#########.###.###.###.#.###
#.......#...#...#.#.....#.....#...#.......#...#.....#...........#.#.#.....#...#.#.#.#.........#...#.#.#.....#...#.........#...#.#...#...#.#.#
#.#.#####.###.###.#####.#.#####.#####.###.#####.###############.###.#######.#.#.#.#.#.###########.#.###.#################.###.#.#.#######.#.#
#.#...#...#...#.#...#...#.#.#...#.....#...#.....#.............#...#.#.......#.....#.#.#.........#.#.....#...............#.#.......#.....#.#.#
#.###.#.###.###.###.#.###.#.#.#####.###.###.#######.#######.#####.#.#.#####.#.#####.#.#####.#####.#######.###.#######.###.#.###.#.#.###.#.#.#
#.#.#.#...#...#...#.#.....#.#...#...#...#...#.....#...#.....#.......#.#.....#.....#.#...#...#...#.#...#...#...#.....#.#...#.....#.....#.#.#.#
#.#.#.#.#####.#.###.#.#####.###.#.###.#####.#.#.#.###.#.#####.#####.#.#.###.###.#.#.###.#.###.#.#.#.#.#.#.#####.###.#.#.#.###########.#.#.#.#
#.......#...#.#.....#.#.......#.#.........#.....#.#.#.#.#.....#...#.#.#.........#.#.....#.....#.....#.#.#.......#.#.#...#.#.......#.#.#...#.#
#.#.#####.#.#.#.#####.#.###.###.#########.#######.#.#.#.#.#####.#.#.#.###.#######.#.###.#.#####.#####.#.#########.#.#.#.###.#####.#.#.#####.#
#.#.....#.#...#.....#...#...#...#...#...#.........#...#.....#...#.#.#...#...#.....#.#...#.#...#.....#.#...........#.#.#.........#.#...#.....#
#.###.#.#.#########.#######.#.###.#.###.###.#########.#####.#.###.#####.###.#.#####.#.###.#.#.#.###.#.#.#########.#.###########.#.#.#####.#.#
#.....#.#.#...#...#.......#.#.....#.#.....#.#...........#.....#.#.....#...#...#.....#.#.#.#.#.#...#...#.#.......#.#.......#...#.#.#.....#.#.#
#########.#.#.#.#.#####.#.#.#######.#.#####.#.#########.#.###.#.#####.###.#####.#.###.#.#.###.###.###.###.#####.#.###.#.###.#.###.#####.###.#
#.........#.#...#.....#...#.......#.#.....#.#.#.....#.#.#.....#...#.#.#...#...#.#.#...#.#.#.....#...#.#...#.....#.#.....#...#.....#...#.#...#
#.#########.#######.#.#.###.#####.#.#####.#.#.###.#.#.#.###.###.#.#.#.#.###.#.#.#.#.###.#.#.###.###.#.#.###.#.#####.#####.#######.#.#.#.#.#.#
#.....#...#.....#.#.#.#.#.....#.#.#.........#.#...#...#...#.#...#...#.#.#...#.....#...#.#...#.#.#.#.#.#...#.#.#.....#.....#...#...#.#.#...#.#
#####.#.#######.#.#.###.###.#.#.#.#.###.#.###.#.###.#.###.#.#.#####.#.#.#.#.#####.###.#.#####.#.#.#.#.###.#.#.###.#.#.#####.#.#####.#.#####.#
#.....#...#.....#.#.......#.#.#...#...#...#...#.....#...#.#...#.....#...#.#...#...#.#.#.....#.....#.....#.#.#.....#.#.#...#.................#
#.#######.#.#####.###.#.#.#.#.###.###.###.#.#######.#####.#####.#############.#.#.#.#.###.###.#########.#.#.#.#####.#.#.#.#.###########.#.#.#
#.#.......#...#.....#.....#.#...#...#.#...#.....#.......#.....#.#...#.....#...#.....#...#.......#...#.#.....#.#...#.#.#.#...................#
#.###.###.###.#.###.###########.###.###.#######.#.###.#.#.#####.#.###.#.#.#.###.#######.#.#######.#.#.#####.#.#.#.###.#.#####.#####.#####.###
#...........#.#...#.....#.......#.....#.......#.#.....#.#.#.....#...#.#.#...#.....#.....#.#...#...#.#...#...#.#.#.#...#.....#.............#.#
#.#.#####.#.#.###.#####.#.###########.#.#.###.#.#######.#.#.#######.#.#.#####.#####.#####.###.#.###.#.###.#.#.#.#.#.#.###.#.###.#.###.#####.#
#.#.....#...#.....#...#.#.........#.#.#.#...........#...#.#.......#...#...#.....#...#...#.....#.#...#...#...#.#.#.#.#.#.....#...#.#.#.#.....#
#.###.#.###########.###.#.#######.#.#.###.#.#######.#.#.#.#######.#######.#.#####.###.#.#######.#.###.#.#.#.#.#.#.#.###.#####.#.#.#.#.#.###.#
#...#.#.#.....#.....#...#...#...#.#.......#.#.....#...#.#.......#.......#.#...#...#...#.........#.#.#.#.#.#...#.#.#...........#.#.#.#.#...#.#
###.#.#.#.#.#.#.#.###.#######.#.#.###.#.#####.#.#######.#######.#######.#.#####.#####.###########.#.#.#.#.#####.#.###############.#.#.#.###.#
#.....................#.......#.#...#...#.....#.......#.......#.......#.#.......#...#...........#.#...#.#.#.#...#.....#...#.......#.#.#.#...#
#.#########.###.#######.#######.###.#####.#########.#.#######.#.#######.#.#######.#.#######.###.#.#.#.#.#.#.#.#######.#.#.#.#######.#.#.#.###
#.......#...#...#.....#.#.....#.....#.....#...#.....#.#.....#.#.........#.........#.....#.......#.#.#...#...#.#.......#.#.#.#.#.....#.#.#...#
#.#####.#.###.###.###.#.#.###.#####.#.#####.###.#######.###.#.#######.#####.###########.#.#######.#.#.#####.#.#.#.###.#.#.#.#.#.###.#.#####.#
#.#...#...........#.#...#.#.....#...#.......#.......#...#...#...#.....#.....#.......#.....#.....#.#.#.......#.#.#.....#.#.#.#.....#...#.....#
#.#.#.#.#.#####.###.#####.#######.#####.###.#.###.#.#.###.#.#.#.###.#.###.###.#####.#########.#.#.###.#######.#.#.#####.#.#.#########.#.###.#
#.#.#...#...#...#.......#.#.....#...#.....#.#...#.#.#.....#.#...#...#...#.#...#...#.#...#.....#.#...#.#.#...#...#.....#.#.#.........#...#.#.#
###.#.#####.#.###.###.#.#.#.###.###.#.#####.###.#.#.###.#.###.###.#####.#.#.###.#.#.#.#.###.#######.#.#.#.#.###.#####.#.#.#.#######.#####.#.#
#...#.#...#...#...#.#.#.#.#...#...#...#...#.#.....#...#.#.....#.....#...#.#.#...#...................#.#.#.#.....#...#.#.#.#.......#.#.#...#.#
#.#.###.#.#####.###.#.#.#.#.#.###.#.#.#.#.#.#.#####.#.#.#########.#.#.#####.#.#.#########.###.#.#####.#.#.#######.###.#.#.#####.###.#.#.#.#.#
#.#.#...#.#.#...#...#.#...#.....#.......#.#.......#.#.#.#...#...#.#.#.......#.#.#.......#...#.#.#...#...#...#.........#.#.....#.#...#...#...#
#.###.###.#.#.###.#.#.#######.#########.#.#.#.###.#.###.#.#.#.#.###.#########.###.#####.###.#.#.#.#.#######.#.#########.#####.###.###.#######
#...#...#.#...#...#.#.#.......#.....#.............#...#...#...#...#...#.....#.....#...#...#.#.#...#...#.....#.#.......#.#.....#...#...#...#.#
#.#.#.###.#.#######.#.#.#######.###.#.###.#.#.#######.###########.###.#####.#.#####.#.###.#.#.###.###.#.#####.#####.#.#.#.#.###.#####.#.#.#.#
#.#...#...#.........#...#.....#...#...#.#...#.....................#...#.....#...#...#.#...#.#.......#.#.....#...#...................#...#...#
#.#####.#########.#####.#.###.###.#####.###.#.###.#.#.#####.#.#.###.###.###.###.#####.#.###.#.#######.#####.###.#.###.###.###.#####.#######.#
#.....#...#.......#...#...#.....#.#.......#.......#.#.#.......#.....#...#.....#.#.....#...#.#.......#.....#...#...#...#...#.#...#...#.#.....#
#####.###.#.#######.#.#####.#####.#.#.###.#.#.#####.#.###.###########.#########.#.###.###.#.#.#####.#####.#.#.#####.###.#.#.###.#.###.#.#####
#...#.#...#.#...#...#.....#...#...#.#.#.#...........#...#.#.......#...........#...#.#.#.#...#.#...#.....#.#.#.#...#.#...#.....#.#.#.....#...#
#.###.#.#####.#.#.#######.#####.#####.#.###.#######.###.###.#####.#.#########.#####.#.#.#####.###.#.#####.###.#.###.#.###.#.###.#.#.#####.#.#
#S......................#.............#...............#.........#...........#.........#...........#...........#.......#...#.......#.......#.#
#############################################################################################################################################
"""

In [4]:
part_1(maze_input)

122492

In [5]:
 maze, best_tiles = part_2(maze_input)
# Mark tiles that are on best paths with 'O'
maze_list = [list(row) for row in maze]
for r, c in best_tiles:
    if maze_list[r][c] == '.':
        maze_list[r][c] = 'O'
    elif maze_list[r][c] in ('S', 'E'):
        # S and E remain S and E, but we know they are on the path
        pass
count = len(best_tiles)
print(count)
for row in maze_list:
    print("".join(row))

520
#############################################################################################################################################
#.....#OOO......#.#.........#.........#.......#OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO..#...#.....#.......#OOO....#OOOOO#OOO....#OOOOOOOOOOOOOOOOOOE#
#.###.#O#O#####.#.#.#####.###.#####.#.#.#####.#O#########O#O###.#.###.#####.#O#.#.###.#.###.#.#.#.#O#O###.#O###O#O#O###.#O#########.#####.###
#...#OOO#O..#.....#.....#.#...#.....#.#OOOOOOOOOOOOOOOOOOOOO..#..............O..#.....#.....#.#...#O#O#...#O#.#OOO#O#...#O#...#.........#...#
###.#O###O#.#.#########.#.#.###.#####.#O#.#.#.#.###.#.###.###.###.#.#.#.#.#.#O#######.###.###.#####O#O#####O#.#####O###.#O#.#.###.#.###.###.#
#...#OOO#OOOOOOOOOOOOOOOOOOOOOOOOO#...#O#.#...#...#.#.......#.#...#...#.#.#.#OOOOO#OOOOO#...#OOOOO#O#OOOOOOO......#OOO#.#O#.#...#.#...#...#.#
#.#####O#.#.#####.#.###.#####.###O#.###O#.#.###.#.#.#.###.###.#.#.#.###.#.#.#.#.#O#O#.#O#####O#O#O#O###########.#####O###O#.###.#.###.#####.#
#.